# Лекция 7. Анализ методических погрешностей для методов укрупнения состояний и агрегирования марковских моделей

 - Анализ погрешностей с использованием параметров связи
 - Анализ погрешностей на основе принципа эквивалентности потоков
 - Метод укрупнения состояний марковских моделей
 - Управление ресурсами в многопроцессорных системах. Алгоритмы Джонсона, Макнотона

## Анализ погрешностей с использованием параметров связи

За основу возьмем задачу из Лекции 6:

<img src="dia1.jpeg">

Для начала найдем точное решение, составив уравнения Колмогорова:

\begin{cases}
  \frac{dP_{200}(t)}{dt}=-2\lambda\cdot P_{200}(t)+p\cdot \nu\cdot P_{110}(t)+\mu\cdot P_{101}(t)\\
  \frac{dP_{110}(t)}{dt}=-(\lambda+ \nu)\cdot P_{110}(t)+\mu\cdot P_{011}(t)+2p\nu\cdot P_{020}(t)+2\lambda\cdot P_{200}(t)\\
  \frac{dP_{020}(t)}{dt}=\lambda\cdot P_{110}(t)-2\nu\cdot P_{020}(t)\\
  \frac{dP_{101}(t)}{dt}=(1-p)\nu\cdot P_{110}(t)+p\nu\cdot P_{011}(t)+2\mu\cdot P_{002}(t)-(\lambda+\mu)\cdot P_{101}(t)\\
  \frac{dP_{011}(t)}{dt}=2(1-p)\nu\cdot P_{020}(t)+\lambda\cdot P_{101}(t)-(\mu+\nu)\cdot P_{011}(t)\\
  \frac{dP_{002}(t)}{dt}=-2\mu\cdot P_{002}(t)+(1-p)\nu\cdot P_{011}(t)\\
  P_{200}(t)+P_{110}(t)+P_{020}(t)+P_{101}(t)+P_{011}(t)+P_{002}(t)=1
\end{cases}

Зададимся некоторыми конкретными значениями $\lambda=0.8, \mu=1.2, \nu=1, p=0.2$ и найдем финальные вероятности, решив СЛАУ:

In [1]:
lambda: 0.8$ mu: 1.2$ nu:1$ p:0.2$

In [10]:
solve([-2*lambda*P_2_0_0+p*nu*P_1_1_0+mu*P_1_0_1=0,
       -(lambda+nu)*P_1_1_0+mu*P_0_1_1+2*p*nu*P_0_2_0+2*lambda*P_2_0_0=0,
       lambda*P_1_1_0-2*nu*P_0_2_0=0,
       (1-p)*nu*P_1_1_0+p*nu*P_0_1_1+2*mu*P_0_0_2-(lambda+mu)*P_1_0_1=0,
       -2*mu*P_0_0_2+(1-p)*nu*P_0_1_1=0,
       P_2_0_0+P_1_1_0+P_0_2_0+P_1_0_1+P_0_1_1+P_0_0_2=1],
       [P_2_0_0,P_1_1_0,P_0_2_0,P_1_0_1,P_0_1_1,P_0_0_2]);


rat: replaced 1.2 by 6/5 = 1.2

rat: replaced 0.2 by 1/5 = 0.2

rat: replaced -1.6 by -8/5 = -1.6

rat: replaced 1.2 by 6/5 = 1.2

rat: replaced 0.4 by 2/5 = 0.4

rat: replaced -1.8 by -9/5 = -1.8

rat: replaced 1.6 by 8/5 = 1.6

rat: replaced 0.8 by 4/5 = 0.8

rat: replaced 2.4 by 12/5 = 2.4

rat: replaced 0.2 by 1/5 = 0.2

rat: replaced -2.0 by -2/1 = -2.0

rat: replaced 0.8 by 4/5 = 0.8

rat: replaced -2.4 by -12/5 = -2.4

rat: replaced 0.8 by 4/5 = 0.8


                   9             72             144             48
(%o10) [[P_2_0_0 = --, P_1_1_0 = ---, P_0_2_0 = ----, P_1_0_1 = ---, 
                   49            245            1225            245
                                                         192              64
                                               P_0_1_1 = ----, P_0_0_2 = ----]]
                                                         1225            1225

In [11]:
float(%);

(%o11) [[P_2_0_0 = 0.1836734693877551, P_1_1_0 = 0.2938775510204082, 
P_0_2_0 = 0.1175510204081633, P_1_0_1 = 0.1959183673469388, 
P_0_1_1 = 0.156734693877551, P_0_0_2 = 0.05224489795918368]]

Найдем основные характеристики системы:
- **среднее число заявок в системе**:
$$L_{\text{сист}}=1\cdot (P_{110}+ P_{101})+2\cdot (P_{020}+P_{011}+P_{002})\approx 1.14285714285714$$

 - **абсолютная пропускная способность** (среднее число заявок, обслуженных в единицу времени): для определения абсолютной пропускной способности учтем, что заявки-программы, в которых обнаружен вирус не получают полного обслуживания (отклоняются), т.е. нас интересует среднее число заявок получивших полное обслуживание в единицу времени. Для замкнутых систем абсолютная пропускная способность выражается как
 $$\lambda'=P_{\text{зан}}\cdot \mu$$
 $P_{\text{зан}}$ - вероятность, что канал занят;
 $\mu$ - интенсивность обслуживания канала.
 
 В нашем случае на выходе два канала, но они работают не во всех состояниях, а только в $S_{101}$ с интенсивностью $\mu$, в состоянии $S_{011}$ тоже с интенсивностью $\mu$, и в состоянии $S_{002}$ с интенсивностью $2\mu$.
 $$\lambda'=\mu\cdot(P_{101}+P_{011})+2\mu\cdot P_{002}\approx 0.548571428571429$$

- *средняя интенсивность суммарного входного потока*:
 $$\Lambda=(k-L_{\text{сист}})\cdot \lambda=(2-1.142857142857143)\cdot 0.8\approx 0.685714285714286$$

- **среднее время пребывания заявки в системе**:
$$T_{\text{сист}}=\frac{L_{\text{сист}}}{\Lambda}\approx 1.666$$

Проведем моделирование.

In [2]:
lambda<-0.8; mu<-1.2; nu<-1; p<-0.2

In [3]:
proger<- rexp(2, lambda) #Время написания программ для двух программистов
virus<-c(0,0,0,0) #первый индекс - номер программиста, от которого проверка на вирус на первой ЭВМ, второй индекс - время обслуживания на 1 ЭВМ
                  #третий индекс -номер програмиста, от которого проверка на вирус на второй ЭВМ, четвертый индекс - время обслуживания на 2 ЭВМ
server<-c(0,0,0,0) #первый индекс - номер программиста, от которого выполнение на первом сервере, второй индекс - время обслуживания на 1 сервере
                  #третий индекс номер программиста, от которого выполнение на втором сервере, четвертый индекс - время обслуживания на 2 сервере
T<-10000 #Суммарное модельное время
h<-0.001 #Шаг приращения модельного времени
t<-0 # Текущее время

In [10]:
obs<-0 #Количество заявок, получивших полное обслуживание
request_col<-0 #Суммарное количество квантов времени нахождения всех заявок в системе
tsum<-0 # Суммарное время пребывания всех заявок в системе
requestsum<-0 # Считаем суммарное количество заявок, поступивших в СМО

In [5]:
while(t<T){
  if ((proger[1]<= t) & (proger[1] !=0))
      {
      requestsum<-requestsum+1
      if (virus[1] == 0)
          {virus[1]<-1
           virus[2]<-t+rexp(1,nu)
           proger[1]<- 0
          }
      else
          {if (virus[3] == 0)
          {virus[3]<-1
           virus[4]<-t+rexp(1,nu)
           proger[1]<- 0
          }}  
      }
  if ((proger[2]<= t) & (proger[2] !=0))
      {
      requestsum<-requestsum+1
      if (virus[1] == 0)
          {virus[1]<-2
           virus[2]<-t+rexp(1,nu)
           proger[2]<- 0
          }
      else
          {if (virus[3] == 0)
          {virus[3]<-2
           virus[4]<-t+rexp(1,nu)
           proger[2]<- 0
          }}  
      }
  if ((virus[2]<=t) &(virus[2] !=0)){
      u<-runif(1)
      if (u > p){
          if (server[1] == 0){
                server[1]<-virus[1]
                server[2]<-t+rexp(1,mu)
                virus[1]<-0;virus[2]<-0;
              }
          else if (server[3] == 0){
                server[3]<-virus[1]
                server[4]<-t+rexp(1,mu)
                virus[1]<-0;virus[2]<-0;
              }
          
          }
      else {
          proger[virus[1]]<-t +rexp(1,lambda)
          virus[1]<-0;virus[2]<-0;
       }
  }
  if ((virus[4]<=t) & (virus[4]!=0)){
      u<-runif(1)
      if (u > p){
          if (server[1] == 0){
                server[1]<-virus[3] 
                server[2]<-t+rexp(1,mu)
                virus[3]<-0;virus[4]<-0;
              }
          else if (server[3] == 0){
                server[3]<-virus[3]
                server[4]<-t+rexp(1,mu)
                virus[3]<-0;virus[4]<-0;
              }
          
          }
      else {
          proger[virus[3]]<-t +rexp(1,lambda)
          virus[3]<-0;virus[4]<-0;
       }
  }
  if ((server[2]<=t) & (server[2] !=0)){
      proger[server[1]]<-t +rexp(1,lambda)
      server[1]<-0;server[2]<-0;
      obs<-obs+1
  }
  if ((server[4]<=t) & (server[4]!=0)){
      proger[server[3]]<-t +rexp(1,lambda)
      server[3]<-0;server[4]<-0;
      obs<-obs+1
  }
  request_col<-request_col+length(proger[proger==0])
  tsum<-tsum+length(proger[proger==0])*h
  t<-t+h
}

In [6]:
#Среднее число заявок в системе
request_col/(T/h)

[1] 1.148258

In [7]:
#Абсолютная пропускная способность
obs/(T)

[1] 0.5515

In [8]:
#Среднее время пребывания заявки в системе
tsum/requestsum

[1] 1.678494

In [9]:
request_col
requestsum

[1] 11482577

[1] 6841

Пробуем теперь использовать метод агрегирования на основе параметров связи.

Для первой модели параметр связи со второй моделью $\mu_{\text{общ}}$ идеально было бы взять равным $\lambda'=0.54$. Но допустим мы этого не знаем и берем $\mu_{\text{общ}}=1$. Конечно, его нужно соизмерять с $\mu$ и количеством каналов, априори $\mu_{\text{общ}}<2\mu$.

Тогда на основе формул (1)-(4) лекции 6 находим:
$$P_0=\left(1+\frac{K\lambda}{\mu_{\text{общ}}}+\frac{K(K-1)\lambda^2}{\mu_{\text{общ}}^2}+\ldots+\frac{K!\lambda^K}{\mu_{\text{общ}}^K}\right)^{-1}=\left(1+\frac{2\lambda}{\mu_{\text{общ}}}+\frac{2\lambda^2}{\mu_{\text{общ}}^2}\right)^{-1}\approx 0.258$$
$$P_1=P_0\frac{2\lambda}{\mu_{\text{общ}}}\approx 0.4128$$
$$P_2=P_0\frac{2\lambda^2}{\mu_{\text{общ}}^2}\approx 0.33$$
$$\ldots$$
$$L_{\text{сист}}=\sum_{k=1}^K k\cdot P_k=1\cdot 0.4128+2\cdot 0.33\approx 1.07$$

Конечно, для второй модели оптимально было бы взять $L_\text{сист}=1.14$, но у нас получилось, как получилось. Переходим ко второй модели.

$$\nu_n=(1-p)\nu\cdot \min{(N,L_{\text{сист}}-n)}, n=0,\ldots,L_{\text{сист}}-1$$
$$\mu_n=\mu\cdot \min{(M, n)}, n=1,2, \ldots,L_{\text{сист}}$$

$$\nu_0=(1-p)\nu\min(2,1-0)=(1-p)\nu=0.8\cdot 1=0.8$$
$$\mu_1=\mu\min(2,1)=\mu=1.2$$
$$\pi_0=\left(1+\frac{\nu_0}{\mu_1}\right)^{-1}\approx 0.6$$
$$\pi_1=\pi_0\cdot \frac{\nu_0}{\mu_1}\approx 0.4$$
$$\mu_{\text{общ}}=\sum_{n=1}^{L_{\text{сист}}}\pi_n\cdot \mu_n=\pi_1\cdot \mu_1=1.2\cdot 0.4= 0.48$$

Это уже ближе к $\mu_{\text{общ}}=0.54$. Выполним еще одну итерацию.

$$P_0=\left(1+\frac{2\lambda}{\mu_{\text{общ}}}+\frac{2\lambda^2}{\mu_{\text{общ}}^2}\right)^{-1}\approx 0.1011$$
$$P_1=P_0\frac{2\lambda}{\mu_{\text{общ}}}\approx 0.337$$
$$P_2=P_0\frac{2\lambda^2}{\mu_{\text{общ}}^2}\approx 0.562$$

$$L_{\text{сист}}=\sum_{k=1}^K k\cdot P_k=1\cdot 0.337+2\cdot 0.562\approx 1.461$$

Ясно, что если округлять $L_{\text{сист}}$ до 1, ничего не поменяется и на этом можно остановиться. Но допустим, мы округлим $L_{\text{сист}}$ до двух:

$$\nu_0=(1-p)\nu\min(2,2-0)=(1-p)\nu=0.8\cdot 1\cdot 2=1.6$$
$$\nu_1=(1-p)\nu\min(2,1)=0.8$$
$$\mu_1=\mu\min(2,1)=\mu=1.2$$
$$\mu_2=2\mu=2.4$$
$$\pi_0=\left(1+\frac{\nu_0}{\mu_1}+\frac{\nu_0\nu_1}{\mu_1\mu_2}\right)^{-1}\approx 0.36$$
$$\pi_1=\pi_0\cdot \frac{\nu_0}{\mu_1}\approx 0.48$$
$$\pi_2=\pi_0\cdot \frac{\nu_0\nu_1}{\mu_1\mu_2}\approx 0.16$$
$$\mu_{\text{общ}}=\sum_{n=1}^{L_{\text{сист}}}\pi_n\cdot \mu_n=\pi_1\cdot \mu_1+\pi_2\cdot\mu_2=1.2\cdot 0.48+2.4\cdot 0.16=0.96$$

Видно, что это округление существенно сказалось на погрешности $\mu_{\text{общ}}$, поэтому правильнее было бы остановиться на предыдущем шаге. 

Итак, оценим параметры эффективности системы:

- **среднее число заявок в системе** $L_{\text{сист}}\approx 1.461$
- **абсолютная пропускная способность** $\lambda'=\mu_{\text{общ}}\approx 0.48$
- **среднее время пребывания заявки в системе** $T_{\text{сист}}=\frac{L_{\text{сист}}}{\Lambda}$, где $\Lambda$ расчитывается аналогичным образом $\Lambda=(k-L_{\text{сист}})\cdot \lambda\approx (2-1.461)\cdot 0.8\approx 0.4312$, $T_{\text{сист}}=1.461/0.4312\approx 3.39$

Окончательно, получаем следующие результаты:

|$L_{\text{сист}}$|$L_{\text{сист}}^{\text{связи}}$|$\lambda'$|$\lambda'^{\text{связи}}$|$T_{\text{сист}}$|$T_{\text{сист}}^{\text{связи}}$|
|--|--|--|--|--|--|
|1.14|1.461|0.55|0.48|1.67|3.39|

## Анализ погрешностей на основе принципа эквивалентности потоков

Проведем аналогичный анализ с использованием метода эквивалентности потоков. 

В данном случае мы дожны перебрать все случаи $u=1,2$:


$$\nu_n=(1-p)\nu\cdot \min{(N,u-n)}, n=0,1$$
$$\mu_n=\mu\cdot \min{(M, n)}, n=1,2$$

$$\pi_0=\left(1+\frac{\nu_0}{\mu_1}+\frac{\nu_0\nu_1}{\mu_1\mu_2}+\ldots\right)^{-1}$$
$$\pi_1=\pi_0\cdot \frac{\nu_0}{\mu_1}$$
$$\pi_2=\pi_0\cdot \frac{\nu_0\nu_1}{\mu_1\mu_2}$$
$$\ldots$$
$$\mu(u)=\sum_{n=1}^{u}\mu_n\pi_n$$

Но фактически эти расчеты мы уже выполняли в предыдущем методе (первая итерация уточнения $\mu_{\text{общ}}=0.48$, вторая итерация уточнения $\mu_{\text{общ}}=0.96$):

$$\mu(1)=0.48$$
$$\mu(2)=0.96$$

Переходим к первой модели:

$$P_0=\left(1+\frac{2\lambda}{\mu(1)}+\frac{2\lambda^2}{\mu(1)\mu(2)}\right)^{-1}\approx 0.176$$
$$P_1=P_0\frac{2\lambda}{\mu(1)}\approx 0.587$$
$$P_2=P_0\frac{2\lambda^2}{\mu(1)\mu(2)}\approx 0.235$$
$$\ldots$$
$$L_{\text{сист}}=\sum_{k=1}^2 k\cdot P_k\approx 1.057$$
$$T_{\text{сист}}=\frac{L_{\text{сист}}}{\lambda(K-L_{\text{сист}})}\approx 1.4$$
$$\mu_{\text{ср}}=\sum_{k=1}^2\mu(n)\cdot P_n\approx 0.507$$



Как видно, результаты вполне близки к точным значениям:

|$L_{\text{сист}}$|$L_{\text{сист}}^{\text{поток}}$|$\lambda'$|$\lambda'^{\text{поток}}$|$T_{\text{сист}}$|$T_{\text{сист}}^{\text{поток}}$|
|--|--|--|--|--|--|
|1.14|1.057|0.55|0.507|1.67|1.4|

## Метод укрупнения состояний марковских моделей

Рассмотрим еще раз задачу 4 из лекции 5:

Одноядерный микропроцессор ставит выполнямые задачи в очередь с ограничением не больше двух задач, иногда микропроцессор переключается на выполнение своих задач. Задача, которая выполняется в этот момент, помещается в очередь, в противном случае убивается. Интенсивность потока задач $\lambda$, потока выполнения задач $\mu$, интенсивность перечлючения операционной системы на выполнение своих задач $\nu$, интенсивность выполнения своих задач $\gamma$. Определить возможные состояния системы и финальные вероятности нахождения в них.

Граф состояний:

$S_0$ - нет выполняемых задач, микропроцессор готов выполнять внешние задачи;

$S_1$ - микропроцессор занят выполнением внешней задачи, очереди нет;

$S_2$ - микропроцессор занят выполнением внешней задачи, в очереди одна задача;

$S_3$ - микропроцессор занят выполнением внешней задачи, в очереди две задачи;

$S_4$ - микропроцессор переключился на свои задачи, в очереди нет задач;

$S_5$ - микропроцессор переключился на свои задачи, в очереди одна задача;

$S_6$ - микропроцессор переключился на свои задачи, в очереди две задачи.

<img src="dia3.jpeg">


Вместо детального решения данной задачи, рассмотрим так называемый метод укрупнения состояний. Суть метода состоит в том, что на графе состояний несколько состояний объединяют в одно. Иногда такое укрупнения оказывается полностью эквивалентным исходному графу: система уравнений относительно вероятностей состояний допускает сложение некоторой группы уравнений с вынесением за скобки общих множителей (интенсивностей перехода). В результате в скобках получается сумма вероятностей некоторой группы состояний. Эта группа и объявляется вероятностью макросостояния (укрупненного состояния). Часто прибегают к неполному эквивалентированию, тогда речь идет о квазиэквивалентном укрупнении. Важно отметить, что этот подход касается именно устоявшихся, стационарных состояний.

Попробуем выполним горизонтальное объединение состояний $S_0, S_1, S_2$ в одно укрупненное состояние $W_0$, а также объединение состояний $S_4, S_5, S_6$ в одно укрупненное состояние $W_1$. В этом случае граф состояний получит следующий вид:

<img src="dia4.jpeg">

Исходная система уравнений имеет, как показано в лекции 5, следующий вид:

$$-(\lambda+\nu)P_0+\mu\cdot P_1+\gamma\cdot P_4=0\tag{1}$$
$$\lambda\cdot P_0-(\lambda+\mu+\nu)\cdot P_1+\mu\cdot P_2+\gamma\cdot P_5=0\tag{2}$$
$$\lambda\cdot P_1-(\lambda+\mu+\nu)\cdot P_2+\mu\cdot P_3+\gamma\cdot P_6=0\tag{3}$$
$$\lambda\cdot P_2-(\mu+\nu)\cdot P_3=0\tag{4}$$
$$\nu\cdot P_0-(\lambda+\gamma)\cdot P_4=0\tag{5}$$
$$\nu\cdot P_1+\lambda\cdot P_4-(\lambda+\gamma)\cdot P_5=0\tag{6}$$
$$\nu\cdot P_2+\nu\cdot P_3+\lambda\cdot P_5-\gamma\cdot P_6=0\tag{7}$$

Для укрупненного графа состояний система примет следующий вид:
$$-(\nu+\lambda)\cdot P_0^h+\gamma\cdot P_1^h+\mu\cdot P_3=0\tag{8}$$
$$-\gamma\cdot P_1^h+\nu\cdot P_0^h+\nu\cdot P_3=0\tag{9}$$
$$-(\mu+\nu)\cdot P_3+\lambda\cdot P_0^h=0\tag{10}$$

Таким образом, вместо 7 уравнений (6 линейно независимых) получили только 3 (2 линейно независимых) уравнения. Причем фактически укрупнение состояний сводится к тому, что мы вместо вероятностей находиться отдельно в каждом из $S_0, S_1, S_2$ состояний определяем вероятность $P_0^h$ находиться или в $S_0$, или в $S_1$, или в $S_2$, которая конечно равна сумме вероятностей $P_0+P_1+P_2$. Аналогично, $P_1^h=P_4+P_5+P_6$. Таким образом, получаем уравнения связи с исходной задачей:

$$P_0^h=P_0+P_1+P_2\tag{11}$$
$$P_1^h=P_4+P_5+P_6\tag{12}$$

Заметим, что складывая уравнения (1)+(2)+(3)+(4) получим уравнение (9):

In [9]:
kill(lambda,nu,mu,gamma)$
expand(-(lambda+nu)*P0+mu*P1+gamma*P4+
      lambda*P0-(lambda+mu+nu)*P1+mu*P2+gamma*P5+
      lambda*P1-(lambda+mu+nu)*P2+mu*P3+gamma*P6+
      lambda*P2-(mu+nu)*P3);

(%o14)  P6 gamma + P5 gamma + P4 gamma - P3 nu - P2 nu - P1 nu - P0 nu

Или, складывая (5)+(6)+(7) тоже получаем (9):

In [10]:
expand(nu*P0-(lambda+gamma)*P4+
      nu*P1+lambda*P4-(lambda+gamma)*P5+
      nu*P2+nu*P3+lambda*P5-gamma*P6);

(%o15) (- P6 gamma) - P5 gamma - P4 gamma + P3 nu + P2 nu + P1 nu + P0 nu

Если сложить (8)+(10), то тоже получаем (9).

Однако система (1)-(7) не будет полностью эквивалентна системе (8)-(12), поскольку сопоставляя уравнения (10) и (4) видно, что $P_0^h$ фактически совпадает с  $P_2$, чего по факту быть не может.

Это подтверждают и расчеты.

Расчеты вероятностей $P_0,P_1,\ldots, P_6$ были выполнены в лекции 5:

In [1]:
lambda:1$ nu:0.5$ mu:0.8$ gamma:1.2$
solve([-(lambda+nu)*P0+mu*P1+gamma*P4=0,
      lambda*P0-(lambda+mu+nu)*P1+mu*P2+gamma*P5=0,
      lambda*P1-(lambda+mu+nu)*P2+mu*P3+gamma*P6=0,
      lambda*P2-(mu+nu)*P3=0,
      nu*P0-(lambda+gamma)*P4=0,
      nu*P1+lambda*P4-(lambda+gamma)*P5=0,
      P0+P1+P2+P3+P4+P5+P6=1],
      [P0,P1,P2,P3,P4,P5,P6]);


rat: replaced -1.5 by -3/2 = -1.5

rat: replaced 0.8 by 4/5 = 0.8

rat: replaced 1.2 by 6/5 = 1.2

rat: replaced -2.3 by -23/10 = -2.3

rat: replaced 0.8 by 4/5 = 0.8

rat: replaced 1.2 by 6/5 = 1.2

rat: replaced -2.3 by -23/10 = -2.3

rat: replaced 0.8 by 4/5 = 0.8

rat: replaced 1.2 by 6/5 = 1.2

rat: replaced -1.3 by -13/10 = -1.3

rat: replaced 0.5 by 1/2 = 0.5

rat: replaced -2.2 by -11/5 = -2.2

rat: replaced 0.5 by 1/2 = 0.5

rat: replaced -2.2 by -11/5 = -2.2


             402688        617760        999700        769000         91520
(%o4) [[P0 = -------, P1 = -------, P2 = -------, P3 = -------, P4 = -------, 
             3951293       3951293       3951293       3951293       3951293
                                                        182000        888625
                                                   P5 = -------, P6 = -------]]
                                                        3951293       3951293

In [2]:
float(%);

(%o5) [[P0 = 0.1019129687421307, P1 = 0.1563437588657688, 
P2 = 0.2530057882318522, P3 = 0.1946198371014248, P4 = 0.02316203835048426, 
P5 = 0.04606087171971301, P6 = 0.2248947369886262]]

In [11]:
#P0h
0.1019129687421307+0.1563437588657688+0.2530057882318522;
#P1h
0.02316203835048426+0.04606087171971301+0.2248947369886262;

[1] 0.5112625

[1] 0.2941176

In [3]:
solve([-(lambda+nu)*P0h+gamma*P1h+mu*P3=0, 
       -gamma*P1h+nu*P0h+nu*P3=0,
       P0h+P1h+P3=1],
       [P0h,P1h,P3]);


rat: replaced -1.5 by -3/2 = -1.5

rat: replaced 1.2 by 6/5 = 1.2

rat: replaced 0.8 by 4/5 = 0.8

rat: replaced 0.5 by 1/2 = 0.5

rat: replaced -1.2 by -6/5 = -1.2

rat: replaced 0.5 by 1/2 = 0.5


                               156        5        120
(%o6)                  [[P0h = ---, P1h = --, P3 = ---]]
                               391        17       391

In [4]:
float(%);

(%o7) [[P0h = 0.3989769820971867, P1h = 0.2941176470588235, 
                                                      P3 = 0.3069053708439898]]

Правильно было бы сказать, что такое укрупнение состояний оказалось некорректным. Это произошло из-за неоднородной структуры графа состояний. 

Рассмотрим однородную структуру:

<img src="dia5.jpeg">

Система уравнений: 

$$-(\lambda+\nu)P_0+\mu\cdot P_1+\gamma\cdot P_4=0\tag{13}$$
$$\lambda\cdot P_0-(\lambda+\mu+\nu)\cdot P_1+\mu\cdot P_2+\gamma\cdot P_5=0\tag{14}$$
$$\lambda\cdot P_1-(\lambda+\mu+\nu)\cdot P_2+\mu\cdot P_3+\gamma\cdot P_6=0\tag{15}$$
$$\lambda\cdot P_2-(\mu+\nu)\cdot P_3+\gamma\cdot P_7=0\tag{16}$$
$$\nu\cdot P_0-(\lambda+\gamma)\cdot P_4=0\tag{17}$$
$$\nu\cdot P_1+\lambda\cdot P_4-(\lambda+\gamma)\cdot P_5=0\tag{18}$$
$$\nu\cdot P_2+\lambda\cdot P_5-(\gamma+\lambda)\cdot P_6=0\tag{19}$$
$$\nu\cdot P_3+\lambda\cdot P_6-\gamma\cdot P_7=0\tag{20}$$

In [17]:
lambda:1$ nu:0.5$ mu:0.8$ gamma:1.2$
solve([-(lambda+nu)*P0+mu*P1+gamma*P4=0,
      lambda*P0-(lambda+mu+nu)*P1+mu*P2+gamma*P5=0,
      lambda*P1-(lambda+mu+nu)*P2+mu*P3+gamma*P6=0,
      lambda*P2-(mu+nu)*P3+gamma*P7=0,
      nu*P0-(lambda+gamma)*P4=0,
      nu*P1+lambda*P4-(lambda+gamma)*P5=0,
      nu*P2+lambda*P5-(gamma+lambda)*P6=0,
      P0+P1+P2+P3+P4+P5+P6+P7=1],
      [P0,P1,P2,P3,P4,P5,P6,P7]);


rat: replaced -1.5 by -3/2 = -1.5

rat: replaced 0.8 by 4/5 = 0.8

rat: replaced 1.2 by 6/5 = 1.2

rat: replaced -2.3 by -23/10 = -2.3

rat: replaced 0.8 by 4/5 = 0.8

rat: replaced 1.2 by 6/5 = 1.2

rat: replaced -2.3 by -23/10 = -2.3

rat: replaced 0.8 by 4/5 = 0.8

rat: replaced 1.2 by 6/5 = 1.2

rat: replaced -1.3 by -13/10 = -1.3

rat: replaced 1.2 by 6/5 = 1.2

rat: replaced 0.5 by 1/2 = 0.5

rat: replaced -2.2 by -11/5 = -2.2

rat: replaced 0.5 by 1/2 = 0.5

rat: replaced -2.2 by -11/5 = -2.2

rat: replaced 0.5 by 1/2 = 0.5

rat: replaced -2.2 by -11/5 = -2.2


              2725888        4181760        6767200        11081500
(%o35) [[P0 = --------, P1 = --------, P2 = --------, P3 = --------, 
              35071493       35071493       35071493       35071493
                         619520        1232000        2098000        6365625
                   P4 = --------, P5 = --------, P6 = --------, P7 = --------]]
                        35071493       35071493       35071493       35071493

In [18]:
float(%);

(%o36) [[P0 = 0.07772375130993141, P1 = 0.1192353003050084, 
P2 = 0.1929544316804534, P3 = 0.3159688696457832, P4 = 0.01766448893407532, 
P5 = 0.03512824503935433, P6 = 0.0598206640361732, P7 = 0.1815042490492207]]

Сложим все уравнения (13)-(16) или (17)-(20):

In [21]:
kill(lambda,mu,nu,gamma)$
expand(-(lambda+nu)*P0+mu*P1+gamma*P4+
      lambda*P0-(lambda+mu+nu)*P1+mu*P2+gamma*P5+
      lambda*P1-(lambda+mu+nu)*P2+mu*P3+gamma*P6+
      lambda*P2-(mu+nu)*P3+gamma*P7);

(%o40) P7 gamma + P6 gamma + P5 gamma + P4 gamma - P3 nu - P2 nu - P1 nu
                                                                        - P0 nu

In [22]:
kill(lambda,mu,nu,gamma)$
expand(nu*P0-(lambda+gamma)*P4+
      nu*P1+lambda*P4-(lambda+gamma)*P5+
      nu*P2+lambda*P5-(gamma+lambda)*P6+
      nu*P3+lambda*P6-gamma*P7);

(%o42) (- P7 gamma) - P6 gamma - P5 gamma - P4 gamma + P3 nu + P2 nu + P1 nu
                                                                        + P0 nu

Фактически получаем эквивалентные уравнения:
$$\gamma\cdot (P_4+P_5+P_6+P_7)-\nu\cdot (P_0+P_1+P_2+P_3)=0$$
$$-\gamma\cdot (P_4+P_5+P_6+P_7)+\nu\cdot (P_0+P_1+P_2+P_3)=0$$

Это соответствует простейшему графу состояний:

<img src="dia6.jpeg">

$$\gamma\cdot P_1^h-\nu\cdot P_0^h=0$$
$$P_0^h+P_1^h=1$$
$$P_0^h=P_0+P_1+P_2+P_3$$
$$P_1^h=P_4+P_5+P_6+P_7$$

Убеждаемся в полной эквивалентности укрупненных состояний:

In [27]:
lambda:1$ nu:0.5$ mu:0.8$ gamma:1.2$
solve([gamma*P1h-nu*P0h=0,P0h+P1h=1],[P0h,P1h]);


rat: replaced -0.5 by -1/2 = -0.5

rat: replaced 1.2 by 6/5 = 1.2


                                    12        5
(%o52)                      [[P0h = --, P1h = --]]
                                    17        17

In [28]:
float(%);

(%o53)      [[P0h = 0.7058823529411765, P1h = 0.2941176470588235]]

In [18]:
float(%);

(%o36) [[P0 = 0.07772375130993141, P1 = 0.1192353003050084, 
P2 = 0.1929544316804534, P3 = 0.3159688696457832, P4 = 0.01766448893407532, 
P5 = 0.03512824503935433, P6 = 0.0598206640361732, P7 = 0.1815042490492207]]

In [29]:
0.07772375130993141+0.1192353003050084+0.1929544316804534+0.3159688696457832;

(%o54)                        0.7058823529411764

Таким образом, такое укрупнение состояний для данной симметричной структуры полностью эквивалентно исходной задаче.  Для нахождения основных характеристик системы можно попробовать выполнить вертикальное укрепнение состояний:
$$P_0^v=P_0+P_4, P_1^v=P_1+P_5, P_2^v=P_2+P_6, P_3^v=P_3+P_7$$

<img src="dia7.jpeg">


Сложим (13), (17) уравнения:

$$-(\lambda+\nu)P_0+\mu\cdot P_1+\gamma\cdot P_4=0$$
$$\nu\cdot P_0-(\lambda+\gamma)\cdot P_4=0$$

$$-\lambda\cdot P_0+\mu\cdot P_1-\lambda\cdot P_4=0\Rightarrow \lambda\cdot P_0^v=\mu\cdot P_1$$

Сложим (14),(18):
$$\lambda\cdot P_0-(\lambda+\mu+\nu)\cdot P_1+\mu\cdot P_2+\gamma\cdot P_5=0$$
$$\nu\cdot P_1+\lambda\cdot P_4-(\lambda+\gamma)\cdot P_5=0$$

$$\lambda\cdot(P_0+P_4)-\lambda\cdot(P_1+P_5)+\mu\cdot(P_2-P_1)=0\Rightarrow \lambda\cdot P_0^v-\lambda\cdot P_1^v+\mu\cdot(P_2-P_1)=0$$

Сложим (15), (19):
$$\lambda\cdot P_1-(\lambda+\mu+\nu)\cdot P_2+\mu\cdot P_3+\gamma\cdot P_6=0$$
$$\nu\cdot P_2+\lambda\cdot P_5-(\gamma+\lambda)\cdot P_6=0$$

$$\lambda\cdot (P_1+P_5)-\lambda\cdot(P_2+P_6)+\mu\cdot(P_3-P_2)=0\Rightarrow \lambda\cdot P_1^v-\lambda\cdot P_2^v+\mu\cdot(P_3-P_2)=0$$

Сложим (16), (20):
$$\lambda\cdot P_2-(\mu+\nu)\cdot P_3+\gamma\cdot P_7=0$$
$$\nu\cdot P_3+\lambda\cdot P_6-\gamma\cdot P_7=0$$
$$\lambda\cdot(P_2+P_6)-\mu\cdot P_3=0\Rightarrow \lambda\cdot P_2^v-\mu\cdot P_3=0$$

Т.е. окончательно получаем следующую систему уравнений:
\begin{cases}
  \lambda\cdot P_0^v=\mu\cdot P_1\\
  \lambda\cdot P_0^v-\lambda\cdot P_1^v=\mu\cdot(P_1-P_2)\\
  \lambda\cdot P_1^v-\lambda\cdot P_2^v=\mu\cdot(P_2-P_3)\\
  \lambda\cdot P_2^v=\mu\cdot P_3
\end{cases}

Получаем следующие условия эквивалентирования:
$$\frac{P_0^v}{P_1}=\frac{P_0^v-P_1^v}{P_1-P_2}=\frac{P_1^v-P_2^v}{P_2-P_3}=\frac{P_2^v}{P_3}=\frac{\mu}{\lambda}$$

Конечно, непосредственно их выполнить не удается, ведь нам неизвестны $P_1, P_2, P_3$.

Рассмотрим более простой случай, в котором все полностью симметрично:

<img src="dia8.jpeg">

$$-(\lambda+\nu)\cdot P_0+\mu\cdot P_1+\gamma\cdot P_2=0\tag{21}$$
$$-(\mu+\nu)\cdot P_1+\gamma\cdot P_3+\lambda\cdot P_0=0\tag{22}$$
$$-(\gamma+\lambda)\cdot P_2+\nu\cdot P_0+\mu\cdot P_3=0\tag{23}$$
$$-(\mu+\gamma)\cdot P_3+\lambda\cdot P_2+\nu\cdot P_1=0\tag{24}$$

Сложим (21), (23), или (22), (24):

$$\lambda\cdot(P_0+P_2)=\mu\cdot(P_1+P_3)\Rightarrow \lambda\cdot P_0^v=\mu\cdot P_1^v\tag{25}$$

Если же мы уберем интенсивность $\mu$ между $S_2, S_3$, то (23) уравнение будет иметь вид:
$$-(\gamma+\lambda)\cdot P_2+\nu\cdot P_0=0\tag{26}$$

Тогда складывая (21) и (26), получим:
$$\lambda\cdot(P_0+P_2)=\mu\cdot P_1$$
Значит нужно снижать $\mu$ до некоторого значения $\mu_1$, чтобы 
$$\lambda\cdot P_0^v = \mu_1\cdot P_1^v$$

На практике для квазиэквивалентности берут $\mu_1=P_0^h\cdot \mu$ - снижается на долю времени, в которой система работоспособна.

## Управление ресурсами в многопроцессорных системах. Алгоритмы Джонсона и Макнотона.

В лекции 6 были рассмотремы вопросы управления ресурсами в однопроцессорной системе. Однако в настоящее время не менее важными оказываются подобные задачи и для многопроцессорных систем.

Здесь можно говорить о двух ограничениях в постановках таких задач: 
- заявки должны пройти последовательную обработку на каждом процессоре (многофазовая обработка);
- заявки могут выполняться на любом из процессоров параллельно.
Исходными данныим для решения этих задач является известная длительность обработки на каждом процессоре.

Рассмотрим сначала последовательную обработку для двухпроцессорной системы (такая постановка известна как задача Джонсона для двух станков).

Для наглядности будем использовать конкретные значения:

|Заявка| 1 | 2 | 3 | 4 | 5 |
|------|---|---|---|---|---|
|Процессор 1|5 |2| 4 | 5 | 6|
|Процессор 2 |7 | 4| 3| 2| 3 |

Из таблицы видно, что, например, первая заявка потребует 5 единиц времени на первом процессоре и 7 единиц времени на втором.

Если выполнять заявки в той последовательности, как они представлены в таблице, то порядок выполнения всех задач будет таким:

![image](dia9.jpeg)

Из рисунка видно, что на первом процессоре заявки выполняются одна за одной без перерывов, для второго процессора, например, вторая заявка не может сразу же начать обрабатываться на втором процессоре после окончания обработки на первом, так как в этот момент на втором процессоре обрабатывается первая заявка. Также между обработкой четвертой и пятой заявок на втором процессоре возникает пауза, т.к. длительность обработки пятой детали на первом процессоре превышает момент готовности второго процессора.

Среднее время ожидания для заявок складывается из ожидания обработки на втором процессоре:

1 заявка выполняется без ожиданий;

2 заявка выполняется с ожиданием 12-7 = 5

3 заявка выполняется с ожиданием 16-11 = 5

4 заявка выполняется с ожиданием 19 - 16 = 3

5 заявка выполняется с ожиданием 22-21 = 1

Среднее время ожидания составило: $\frac{0+5+5+3+1}{5}\approx 2.8$

Суммарное время выполнения всех заявок составило 25 единиц времени. Вопрос: можно ли закончить обработку всех заявок быстрее?

Ответ на этот вопрос дает **алгоритм Джонсона** - он заключается в следующем:
 - из всех длительностей обслуживания выбирается самая минимальная. Если эта длительность относится к первому процессору, то такая заявка выполняется первой, если эта длительность относится ко второму процессору, то такая заявка выполняется последней.
 - найденную заявку с ее длительностями удаляют из рассмотрения, предыдущий пункт повторяют.

Рассмотрим использование этого алгоритма на нашем примере:

|Заявка| 1 | 2 | 3 | 4 | 5 |
|------|---|---|---|---|---|
|Процессор 1|5 |2| 4 | 5 | 6|
|Процессор 2 |7 | 4| 3| 2| 3 |

Видно, что минимальная продолжительность равна 2 - это 2-я заявка для первого процессора (выполняется первой), 4-я заявка для второго процессора, значит она выполняется последней.

|Порядок| 2 | пусто  | пусто | пусто |  4|
|--|--|--|--|--|--|

После вычеркивания получаем

|Заявка| 1 | 3 | 5|
|------|---|---|---|
|Процессор 1|5 | 4 | 6|
|Процессор 2 |7 | 3| 3 |

Следующее минимальное число - 3, соответствует 3, 5 заявке на втором процесоре. Значит 3 или 5 заявки должны выполняться предпоследними

|Порядок| 2 | пусто  | 3(5) | 5(3) |  4|
|--|--|--|--|--|--|

Ну и оставшуюся первую заявку нужно выполнять второй по счету.

|Порядок| 2 | 1  | 3(5) | 5(3) |  4|
|--|--|--|--|--|--|

Итак, очередность выполнения задач определена. Посмотрим на результаты.

![image](dia10.jpeg)

Таким образом, общая продолжительность уменьшилась (была 25, стала 24 единицы времени).

Время ожидания заявок:
- Первая заявка 7-6=1
- Вторая заявка 2-2=0
- Третья заявка 17-17=0
- Четвертая заявка 22-20=2
- Пятая заявка 14-13=1

Среднее время ожидания $\frac{1+0+0+2+1}{5}\approx 0.8$

Итак, наблюдаем явные выгоды от использования предложенного алгоритма. Рассмотренная задача называется  **задачей двухпроцессорного обслуживания**, или задачей Джонсона (по имени S.M. Johnson, который в 1954 г. предложил алгоритм для её решения).

Алгоритм довольно прост. Но интересным фактом является то, что для большего количества станков алгоритм неизвестен, более того когда число процессоров больше двух, эта задача становится NP-полной (как доказал Гэри (Garey) в 1976 г.).

Рассмотрим теперь подходы к обслуживанию заявок, которые могут выполняться на любом из процессоров параллельно.

Пусть имеется $n$ идентичных процессоров, на которых необходимо решить $L$ независимых задач, для решения каждой задачи на любом процессоре требуется $t_i, i=1,2,\ldots, L$ единиц времени. 

Для простоты опять рассмотрим двухпроцесорную систему с длительностями задач:

|Задачи|Задача 1|Задача 2| Задача 3|Задача 4|Задача 5|
|--|--|--|--|--|--|
| |3|3|2|2|2|

Рассмотрим несколько примеров составления расписания.

![image](dia11.jpeg)

Видно, что последний вариант является наиболее предпочтительным.

Можно предложить нижнюю оценку времени для оптимального расписания. Длина любого расписания не может быть меньше максимального $t_i$, в противном случае мы не выполним задачу с максимальной длительностью. Длина расписания не может также быть меньше времени, когда все процессоры тратят одинаковое время и не простаивают, это соответствует среднему времени выполнения $\frac{\sum_{i=1}^L t_i}{n}$. Таким образом, получаем оценку
$$T_{opt} \geq \max\left\{\max\{t_i\}, \frac{\sum_{i=1}^L t_i}{n}\right\}$$

В общем случае даже при $n=2$ задача поиска оптимального расписания и $T_{opt}$ является $NP$-трудной, т.е. все известные алгоритмы ее решения имеют трудоемкость, экспоненциально зависящую от $L$.

Однако, если допустить возможность прерывания обслуживания и последующее его продолжение, то алгоритмы поиска оптимального расписания полиномиальной сложности существуют. При этом предполагается, что после прерывания задача может быть продолжена на любом свободном процессоре, а необязательно на том, на котором она выполнялась первоначально. Число прерывания должно быть по возможности наименьшим, поскольку с каждым прерыванием расткт потери машинного времени на загрузку-выгрузку задач из оперативной памяти.

Рассмотрим алгоритм, предложенный Макнотоном в 1959 году. 

**Алгоритм Макнотона** строит оптимальное расписание с не более чем $n-1$ прерываниями. Он заключается в предварительном упорядочении задач по убыванию времени решения и назначении задач последовательно по порядку номеров одну за другой на процессоры системы справа налево от значения $\max\left\{\max\{t_i\}, \frac{\sum_{i=1}^L t_i}{n}\right\}$.

Рассмотрим пример: $n=2, L=4, t_1=5, t_2=4, t_3=3, t_4=2$, тогда оценка
$$\max\left\{\max\{t_i\}, \frac{\sum_{i=1}^L t_i}{n}\right\}=\max\left\{5, (5+4+3+2)/2\right\}=7$$

В соответствии с алгоритмом Макнотона вначале назначает на первый процессор первую задачу, отсчитывая вправо от момента времени 7.

![image](dia12.jpeg)

Далее назначаем следующую по длительности вторую задачу, но поскольку ее длительность больше оставшегося на первом процессоре времени, мы вынуждены сделать прерывание, начав загружать второй процессор и опять справа налево ставшейся частью второй задачи.

![image](dia13.jpeg)

Оставшиеся задачи планируем на второй процессор

![image](dia14.jpeg)

Итак, время выполнения совпадает с нижним ограничением, значит оно оптимально, при этом нам потребовалось одно прерывание.

Если попытаться решать задачу без прерываний, то, как было отмечено, она является $NP$-трудной. Один из наиболее эффективных и нетрудоемких алгоритмов организации вычислений в этом случае - это алгоритм LPT (longest-processing task first) - самая длинная задача решается первой. Суть этого алгоритма заключается в назначении задача в порядке убыванния времени решения на освобождающиеся процессоры. 

В 1967 году П. Грэхем (сотрудник Bell Laboratories) получил следующий результат:

**При использовании алгоритма LPT для распределения любого набора задач без прерываний в системе с $n$ идентичными процессорами справедливо
$$T\leq\left(\frac{4}{3}-\frac{1}{3n}\right)T_0$$
где $T_0$ - длина оптимального расписания**